# Scraping with Pandas

In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pymongo
import requests

### NASA Mars News
Scrape the Mars News Site (url) to collect the latest News Titles and Paragraph text. 
Save these to variables to use them later.

Set up URL
Retrieve page with splinter

In [ ]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
url = 'https://redplanetscience.com/'
browser.visit(url)

In [ ]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

sidebar = soup.find('div', class_="col-md-12")
categories = sidebar.find_all('div')

Examine the results, determine element that contains the title and paragraph. 

In [ ]:
#print(categories)

In [ ]:
titles = []
paragraphs = []

for category in categories:
    title = getattr(category.find('div', class_='content_title'),'text',None)
    titles.append(title)
    paragraph = getattr(category.find('div', class_='article_teaser_body'),'text',None)
    paragraphs.append(paragraph)
    if (title and paragraph):
    # Print results
        print('-------------')
        print(title)
        print(paragraph)

In [ ]:
browser.quit()

### JPL Mars Space Images - Featured Image
Visit the Featured Space Image site (image_url)to collect images.
Use splitner to navigate to the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.
Make sure to find the image url to the full size .jpg image.
Make sure to save a complete url string for this image: for example:
featured_image_url = 'https://spaceimages-mars.com/image/featured/mars2.jpg'

#### set up splinter

In [ ]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
image_url = 'https://spaceimages-mars.com/'
browser.visit(image_url)

In [ ]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
print(soup)

In [ ]:
featured_image_url = soup.find('a', class_="showimg fancybox-thumbs")['href']

In [ ]:
featured_image_url = image_url + featured_image_url
print(featured_image_url)

In [ ]:
browser.quit()

### Mars Facts
Visit the Mars Facts webpage (facts_url) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
Use Pandas to convert the data to a HTML table string.

In [ ]:
facts_url = 'https://galaxyfacts-mars.com/'

In [ ]:
tables = pd.read_html(facts_url)
tables

In [ ]:
#return is a list of dataframes for any tabular data that Pandas found
type(tables)

In [ ]:
#slice off dataframes that we want using normal indexing
facts_df = tables[0]

In [ ]:
#drop single header rows
facts_df.columns = ['Mars - Earth Comparison','Mars','Earth']
facts_df.head()

In [ ]:
facts_df.drop([0], inplace=True)

In [ ]:
facts_df.set_index('Mars - Earth Comparison',inplace=True)

In [ ]:
facts_df

In [ ]:
mars_profile_df = tables[1]
mars_profile_df.columns = ['Mars','Planet Profile']

In [ ]:
mars_profile_df.set_index('Mars',inplace=True)
mars_profile_df

#### Use pandas  `to_html` method to generate HTML tables from DataFrames. 

In [ ]:
facts_html_table = facts_df.to_html()
profile_html_table = mars_profile_df.to_html()

#### Strip unwanted newlines to clean up the table.

In [ ]:
profile_html_table.replace('\n', '')
facts_html_table.replace('\n', '')

### Mars Hemispheres
Visit the astrogeology site (astro_url) to obtain high resolution images for each of Mars's hemispheres.

- Click each of the links to the hemispheres in order to find the image url to the full resolution image.
- Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. 
- Use a Python dictionary to store the data using the keys img_url and title.
- Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [/Users/mallorykanderson/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [3]:
astro_url = 'https://marshemispheres.com/'
browser.visit(astro_url)

In [4]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [5]:
enhanced_urls = []
titles = []

In [6]:
results = soup.find_all('div', class_='item')

In [7]:
#print(results)

In [8]:
#borrowed from Mike H -- because I was getting more content than needed.

for result in results:
    if result.find('h3'):
        title = result.find('h3').text
        titles.append(title)
        print(title)
    if result.find('a'):
        link = result.find('a')
        href = link['href']
        enhanced_url = astro_url + href
        enhanced_urls.append(enhanced_url)
        print(enhanced_url)

Cerberus Hemisphere Enhanced
https://marshemispheres.com/cerberus.html
Schiaparelli Hemisphere Enhanced
https://marshemispheres.com/schiaparelli.html
Syrtis Major Hemisphere Enhanced
https://marshemispheres.com/syrtis.html
Valles Marineris Hemisphere Enhanced
https://marshemispheres.com/valles.html


In [9]:
#iterate over the link list using .click() to pull the high res image url from each link

image_urls = []

for enhanced_url in enhanced_urls:
    browser.visit(enhanced_url)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

   #browser.links.find_by_partial_text('Sample').click()   
    
    link = soup.body.find_all('img', class_='wide-image')
    src = link[0]['src']
    image_url = astro_url + src
    image_urls.append(image_url)
    print(image_url)

https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg
https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg
https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg
https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg


In [10]:
browser.quit()

In [12]:
hemisphere_image_urls = []

In [13]:
#set up the dictionary to be inserted into MongoDb
for i,j in zip(titles,image_urls):
    hemisphere_image_urls.append({"title": i,'image_url': j})

In [14]:
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'image_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'image_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'image_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'image_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]
